In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization ,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import numpy as np
import pandas as pd

In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
train = get_training_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/train')
test = get_training_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/test')
val = get_training_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/val')

In [ ]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

In [ ]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [ ]:
# resize data for deep learning 
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [ ]:
# With data augmentation to prevent overfitting and handling the imbalance in dataset
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
#model.summary()

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

In [ ]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 32) ,epochs = 12 , validation_data = datagen.flow(x_val, y_val),callbacks = [learning_rate_reduction])

In [ ]:
print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

plt.plot(epochs,acc,"r",label="Training Accuracy")
plt.plot(epochs,val_acc,"b",label="Validation Accuracy")
plt.title("Training And Validation Accuracy")

plt.legend()
plt.figure()

plt.plot(epochs,loss,"r",label="Training Loss")
plt.plot(epochs,val_loss,"b",label="Validation Loss")
plt.title("Training And Validation Loss")

plt.legend()
plt.show()

In [ ]:
predictions = model.predict_classes(x_test)
predictions = predictions.reshape(1,-1)[0]
predictions[:15]

In [ ]:
print(classification_report(y_test, predictions, target_names = ['Pneumonia (Class 0)','Normal (Class 1)']))

In [ ]:
cm = confusion_matrix(y_test,predictions)
cm

In [ ]:
cm = pd.DataFrame(cm , index = ['0','1'] , columns = ['0','1'])

In [ ]:
cm

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='',xticklabels = labels,yticklabels = labels)

In [ ]:
x_test.shape

# **第二个数据的加入，取置信度足够高的数据加入模型**

In [ ]:
train_df = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')
train_df.shape

In [ ]:

train_df.head(5)

In [ ]:
t = train_df[train_df.Label_2_Virus_category == 'COVID-19']


In [ ]:
len(t)

In [ ]:
def make_dir():
    try:
        os.makedirs('../working/test/NORMAL')
        os.makedirs('../working/test/PNEUMONIA')
    except:
        pass

In [ ]:
make_dir()

In [ ]:
import shutil

In [ ]:
tt = t.X_ray_image_name.values


In [ ]:
path = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'
source = '../working/test/PNEUMONIA'

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(tt):
    a = os.path.join(path,i)
    shutil.copy(a,source)

In [ ]:
test2 = get_training_data('../working/test')

In [ ]:
test2[:,1]

In [ ]:
x_test2 = []
y_test2 = []

for feature,label in test2:
    x_test2.append(feature)
    y_test2.append(label)

x_test2 = np.array(x_test2) / 255

x_test2 = x_test2.reshape(-1, img_size, img_size, 1)

y_test2 = np.array(y_test2)

In [ ]:
model.evaluate(x_test2,y_test2)

In [ ]:
x=model.predict(x_test2)[:,0]
np.argwhere(x>0.85)

# **工作2**

In [ ]:
train_df = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')
train_df.head(5)
#train_df.shape[0]

In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
train2=[]
test2=[]
img_size = 150
data_dir="../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset"
for i in tqdm(range(train_df.shape[0])):
    a=train_df.X_ray_image_name[i]
    b=train_df.Label[i]
    c=train_df.Dataset_type[i]
    if b=="Normal":
        b=1
    else:
        b=0
    if(c=="TRAIN") :
        c="train"
    elif(c=="TEST"):
        c="test"
    path=os.path.join(data_dir, c)
    img_arr = cv2.imread(os.path.join(path,a), cv2.IMREAD_GRAYSCALE)
    resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
    if c=="train":
        train2.append([resized_arr, b])
    else:
        test2.append([resized_arr, b])
train2=np.array(train2,dtype='object')
test2=np.array(test2,dtype='object')


In [ ]:
x_train2 = []
y_train2 = []
for feature, label in train2:
    x_train2.append(feature)
    y_train2.append(label)
x_train2 = np.array(x_train2) / 255
x_train2 = x_train.reshape(-1, img_size, img_size, 1)
y_train2 = np.array(y_train2)

x_test2 = []
y_test2 = []
for feature, label in test2:
    x_test2.append(feature)
    y_test2.append(label)
x_test2 = np.array(x_test2) / 255
x_test2 = x_train.reshape(-1, img_size, img_size, 1)
y_test2 = np.array(y_test2)
#x_train2,y_train2
#x_test2,y_test2

In [ ]:
x=model.predict(x_train2)
x[:6]
#(x>0.85)&(y_train2==1)
#col=np.argwhere(x>0.85)
#x_train2[col]

In [ ]:

y=model.predict_classes(x_train2)



In [ ]:
alpha=0.05
d=[]
for i in range(x.size):
    if (x[i]>=1-alpha):
        d.append(i)
    if (x[i]<=alpha):
        d.append(i)
d1=x_train2[d]
d2=y[d]

In [ ]:
history = model.fit(datagen.flow(d1,d2, batch_size = 32) ,epochs = 12 ,callbacks = [learning_rate_reduction])

In [ ]:
model.evaluate(x_train,y_train)